In [1]:
import os

In [2]:
os.chdir("../")
print(os.getcwd())

c:\Users\jv3250328\Downloads\End_To_End_Machine_Learning_Project_MLOps


In [3]:
import tensorflow as tf

In [4]:
model = tf.keras.models.load_model("artifacts/training/trained_model.h5")

In [5]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen = True)
class EvaluationConfig:
    path_of_model: Path
    train_data: Path
    all_params: dict
    param_image_size: list
    param_batch_size: int

In [6]:
from End_to_End_ML_Pipeline.constants import *
from End_to_End_ML_Pipeline.utils.common import read_yaml, create_directories, save_json

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:

        eval_config = EvaluationConfig(
            path_of_model = "artifacts/training/trained_model.h5",
            train_data = "artifacts/data_ingestion/Chicken-fecal-images",
            all_params = self.params,
            param_image_size = self.params.IMAGE_SIZE,
            param_batch_size = self.params.BATCH_SIZE
        )

        return eval_config

In [8]:
from urllib.parse import urlparse

In [9]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        )

        dataflow_kwargs = dict(
            target_size = self.config.param_image_size[:-1],
            batch_size = self.config.param_batch_size,
            interpolation = "bilinear"
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self._valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.train_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )
        
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        model = tf.keras.models.load_model(path)
        return model
        
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.scores = self.model.evaluate(self._valid_generator)
            
    def save_score(self):
        scores = {
            "loss": self.scores[0],
            "accuracy": self.scores[1]
        }
        save_json(path = Path("scores.json"), data = scores)

In [10]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(config = val_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
    raise e

Found 116 images belonging to 2 classes.


c:\Users\jv3250328\.conda\envs\chicken\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.5000 - loss: 0.9469
